This notebook contains the code in data scraping.

In [14]:
# Import the dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests

In [15]:
# Setting the url
url = "https://dojos.info/NewJersey/Counties.aspx"

# Creating the response
response = requests.get(url)

print(response)

# Using beautiful soup
soup = bs(response.text, "lxml")

<Response [200]>


In [50]:
# Grabbing the list of counties in New Jersey
state = soup.find_all('ul', class_ = "olist")[0]
state

<ul class="olist">
<li><h4>A</h4>
<ol class="c"><li><a href="AtlanticCounty/" title="Martial Arts Schools in Atlantic County, NJ">Atlantic County</a> (31)</li>
</ol>
</li>
<li><h4>B</h4>
<ol class="c"><li><a href="BergenCounty/" title="Martial Arts Schools in Bergen County, NJ">Bergen County</a> (126)</li>
<li><a href="BurlingtonCounty/" title="Martial Arts Schools in Burlington County, NJ">Burlington County</a> (44)</li>
</ol>
</li>
<li><h4>C</h4>
<ol class="c"><li><a href="CamdenCounty/" title="Martial Arts Schools in Camden County, NJ">Camden County</a> (55)</li>
<li><a href="CapeMayCounty/" title="Martial Arts Schools in Cape May County, NJ">Cape May County</a> (8)</li>
<li><a href="CumberlandCounty/" title="Martial Arts Schools in Cumberland County, NJ">Cumberland County</a> (15)</li>
</ol>
</li>
<li><h4>E</h4>
<ol class="c"><li><a href="EssexCounty/" title="Martial Arts Schools in Essex County, NJ">Essex County</a> (67)</li>
</ol>
</li>
<li><h4>G</h4>
<ol class="c"><li><a href="G

In [51]:
# Pulling the individual counties
county = state.find_all("li", class_ = "a")
county

[]

In [44]:
county[0].text

'A\nAtlantic County (31)\n\n'

In [45]:
len(county)

33

In [46]:

for i in range(len(county)):
    print(county[i].text)

A
Atlantic County (31)


Atlantic County (31)
B
Bergen County (126)
Burlington County (44)


Bergen County (126)
Burlington County (44)
C
Camden County (55)
Cape May County (8)
Cumberland County (15)


Camden County (55)
Cape May County (8)
Cumberland County (15)
E
Essex County (67)


Essex County (67)
G
Gloucester County (30)


Gloucester County (30)
H
Hudson County (43)
Hunterdon County (21)


Hudson County (43)
Hunterdon County (21)
M
Mercer County (38)
Middlesex County (64)
Monmouth County (92)
Morris County (63)


Mercer County (38)
Middlesex County (64)
Monmouth County (92)
Morris County (63)
O
Ocean County (39)


Ocean County (39)
P
Passaic County (45)


Passaic County (45)
S
Salem County (7)
Somerset County (40)
Sussex County (25)


Salem County (7)
Somerset County (40)
Sussex County (25)
U
Union County (46)


Union County (46)
W
Warren County (18)


Warren County (18)
